# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.6.0
Keras Version: 2.6.0

Python 3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 05:59:45) [MSC v.1929 64 bit (AMD64)]
Pandas 1.4.3
Scikit-Learn 1.1.1
GPU is available


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import os

In [3]:
classifier = Sequential()

#classes = 2

#Kernels usually 1x1 | 3x3 | 5x5

#Adding a convolutional layer
classifier.add(Conv2D(8, (3, 3), input_shape = (150, 150, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a second convolutional layer
classifier.add(Conv2D(16, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Flattening
classifier.add(Flatten())

#Full connection
classifier.add(Dense(units = 32, activation = 'relu'))
classifier.add(Dropout(0.30))
classifier.add(Dense(units = 1, activation= 'sigmoid'))

Let's double check the model description:

In [4]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 21904)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                700960    
_________________________________________________________________
dropout (Dropout)            (None, 32)                0

In [5]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.00001)
classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path_dir = '../../dataset/train'
parth_dir_test = '../../dataset/test'

datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    shuffle=True,
    subset='training',
    batch_size = 64,
    seed = 11,
    class_mode = 'binary'
)
validation_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    subset='validation',
    batch_size = 64,
    seed = 11,
    class_mode = 'binary'
)


Found 2202 images belonging to 2 classes.
Found 550 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'nonviolence': 0, 'violence': 1}

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=30)
mc = ModelCheckpoint('best_cnn.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=30)

# Create a TensorBoard callback
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
classifier.fit(train_generator,
                         epochs = 200,
                         callbacks = [tensorboard_callback,es,mc,reduceLR],
                         validation_data = validation_generator
                         )

Epoch 1/200
35/35 [==============================] - 11s 248ms/step - loss: 0.6947 - accuracy: 0.5141 - val_loss: 0.6889 - val_accuracy: 0.5000
Epoch 2/200
35/35 [==============================] - 8s 228ms/step - loss: 0.6870 - accuracy: 0.5563 - val_loss: 0.6815 - val_accuracy: 0.6564
Epoch 3/200
35/35 [==============================] - 8s 232ms/step - loss: 0.6818 - accuracy: 0.5767 - val_loss: 0.6713 - val_accuracy: 0.7327
Epoch 4/200
35/35 [==============================] - 8s 231ms/step - loss: 0.6729 - accuracy: 0.6521 - val_loss: 0.6629 - val_accuracy: 0.7091
Epoch 5/200
35/35 [==============================] - 8s 230ms/step - loss: 0.6630 - accuracy: 0.6798 - val_loss: 0.6524 - val_accuracy: 0.7418
Epoch 6/200
35/35 [==============================] - 8s 228ms/step - loss: 0.6562 - accuracy: 0.6689 - val_loss: 0.6417 - val_accuracy: 0.7091
Epoch 7/200
35/35 [==============================] - 8s 232ms/step - loss: 0.6455 - accuracy: 0.7016 - val_loss: 0.6281 - val_accuracy: 0.767

Epoch 58/200
35/35 [==============================] - 8s 226ms/step - loss: 0.4403 - accuracy: 0.8152 - val_loss: 0.4032 - val_accuracy: 0.8582
Epoch 59/200
35/35 [==============================] - 8s 227ms/step - loss: 0.4332 - accuracy: 0.8161 - val_loss: 0.4030 - val_accuracy: 0.8509
Epoch 60/200
35/35 [==============================] - 8s 231ms/step - loss: 0.4376 - accuracy: 0.8152 - val_loss: 0.4011 - val_accuracy: 0.8618
Epoch 61/200
35/35 [==============================] - 8s 225ms/step - loss: 0.4327 - accuracy: 0.8188 - val_loss: 0.3999 - val_accuracy: 0.8618
Epoch 62/200
35/35 [==============================] - 8s 227ms/step - loss: 0.4359 - accuracy: 0.8179 - val_loss: 0.3989 - val_accuracy: 0.8673
Epoch 63/200
35/35 [==============================] - 8s 226ms/step - loss: 0.4407 - accuracy: 0.8211 - val_loss: 0.3987 - val_accuracy: 0.8727
Epoch 64/200
35/35 [==============================] - 8s 229ms/step - loss: 0.4316 - accuracy: 0.8265 - val_loss: 0.3965 - val_accuracy:

Epoch 115/200
35/35 [==============================] - 8s 227ms/step - loss: 0.3770 - accuracy: 0.8447 - val_loss: 0.3647 - val_accuracy: 0.8382
Epoch 116/200
35/35 [==============================] - 8s 227ms/step - loss: 0.3780 - accuracy: 0.8488 - val_loss: 0.3536 - val_accuracy: 0.8745
Epoch 117/200
35/35 [==============================] - 8s 228ms/step - loss: 0.3692 - accuracy: 0.8629 - val_loss: 0.3518 - val_accuracy: 0.8709
Epoch 118/200
35/35 [==============================] - 8s 229ms/step - loss: 0.3762 - accuracy: 0.8438 - val_loss: 0.3675 - val_accuracy: 0.8273
Epoch 119/200
35/35 [==============================] - 8s 228ms/step - loss: 0.3754 - accuracy: 0.8420 - val_loss: 0.3510 - val_accuracy: 0.8764
Epoch 120/200
35/35 [==============================] - 8s 226ms/step - loss: 0.3718 - accuracy: 0.8497 - val_loss: 0.3524 - val_accuracy: 0.8727
Epoch 121/200
35/35 [==============================] - 8s 227ms/step - loss: 0.3688 - accuracy: 0.8551 - val_loss: 0.3580 - val_ac

35/35 [==============================] - 8s 235ms/step - loss: 0.3306 - accuracy: 0.8692 - val_loss: 0.3324 - val_accuracy: 0.8545
Epoch 172/200
35/35 [==============================] - 8s 235ms/step - loss: 0.3268 - accuracy: 0.8678 - val_loss: 0.3287 - val_accuracy: 0.8582
Epoch 173/200
35/35 [==============================] - 8s 233ms/step - loss: 0.3318 - accuracy: 0.8697 - val_loss: 0.3252 - val_accuracy: 0.8618
Epoch 174/200
35/35 [==============================] - 8s 232ms/step - loss: 0.3256 - accuracy: 0.8742 - val_loss: 0.3251 - val_accuracy: 0.8618
Epoch 175/200
35/35 [==============================] - 8s 232ms/step - loss: 0.3223 - accuracy: 0.8797 - val_loss: 0.3288 - val_accuracy: 0.8582
Epoch 176/200
35/35 [==============================] - 8s 238ms/step - loss: 0.3248 - accuracy: 0.8738 - val_loss: 0.3293 - val_accuracy: 0.8600
Epoch 177/200
35/35 [==============================] - 8s 235ms/step - loss: 0.3270 - accuracy: 0.8756 - val_loss: 0.3237 - val_accuracy: 0.8618

In [10]:
#Overfitting -> Your training accuracy will be higher than the accuracy on the validation/test set

#Overfitting indicates that your model is too complex for the problem that it is solving,
#filters in the case of Convolutional Neural Networks, and layers in the case of overall Deep Learning Models

#How do you know if your model is underfitting? Your model is underfitting if the accuracy on the validation set 
#is higher than the accuracy on the training set. Additionally, 
#if the whole model performs bad this is also called underfitting.

#https://towardsdatascience.com/deep-learning-3-more-on-cnns-handling-overfitting-2bd5d99abe5d

In [11]:
%tensorboard --logdir logs/fit

# Part 3 test the CNN

In [12]:
# Load Model
classifier.load_weights('best_cnn.h5')

In [13]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 21904)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                700960    
_________________________________________________________________
dropout (Dropout)            (None, 32)                0

In [15]:
test_set = datagen.flow_from_directory(parth_dir_test,
                                        target_size= (150,150),
                                        batch_size = 64,
                                        seed = 11,
                                        class_mode = 'binary',
                                        shuffle=False)

score = classifier.evaluate(test_set, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

Found 688 images belonging to 2 classes.
Test Loss 0.34699878096580505
Test accuracy 0.882267415523529


In [16]:
test_set.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [17]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

yhat_classes = classifier.predict_classes(test_set, verbose=0)
yhat_classes = yhat_classes[:, 0]

print(classification_report(test_set.classes,yhat_classes))

C:\Users\bruno\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.89      0.87      0.88       344
           1       0.87      0.89      0.88       344

    accuracy                           0.88       688
   macro avg       0.88      0.88      0.88       688
weighted avg       0.88      0.88      0.88       688

